In [2]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st
import plotly.express as px
import numpy as n

In [3]:
def get_passenger_data():
    # building url
    url = f'https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/road_eqs_carmot?format=JSON&unit=NR&mot_nrg=TOTAL&mot_nrg=PET&mot_nrg=DIE&engine=TOTAL&engine=CC_LT1400&engine=CC1400-1999&engine=CC_GE2000&lang=en'
    url_text = requests.get(url)

    passenger_dict = json.loads(url_text.text)

    return passenger_dict


In [4]:
data = get_passenger_data()
data

{'version': '2.0',
 'class': 'dataset',
 'label': 'Passenger cars, by type of motor energy and size of engine',
 'source': 'ESTAT',
 'updated': '2025-05-15T23:00:00+0200',
 'value': {'24783': 233368,
  '24784': 252775,
  '24785': 277611,
  '23789': 1088000,
  '23790': 1211000,
  '23791': 1325000,
  '23792': 1413000,
  '23793': 1528000,
  '23794': 1642000,
  '23795': 1731000,
  '23796': 1737000,
  '23797': 1780000,
  '23798': 1808000,
  '23799': 1847000,
  '23800': 1895000,
  '23801': 1946000,
  '23802': 1999000,
  '23803': 2040083,
  '23804': 2075423,
  '23805': 2114381,
  '23806': 2161479,
  '23807': 2189040,
  '23808': 2205012,
  '23809': 2218877,
  '23810': 2232277,
  '23811': 2220991,
  '23812': 2189120,
  '23813': 2157405,
  '24505': 411265,
  '24506': 443778,
  '24507': 475081,
  '24508': 502914,
  '24509': 520360,
  '24510': 548988,
  '24511': 573425,
  '24512': 580679,
  '24513': 619808,
  '24514': 641894,
  '24515': 674227,
  '22769': 1865000,
  '22770': 1962000,
  '22771': 20

In [5]:
print(len(data["value"]))
data["dimension"]

6765


{'freq': {'label': 'Time frequency',
  'category': {'index': {'A': 0}, 'label': {'A': 'Annual'}}},
 'unit': {'label': 'Unit of measure',
  'category': {'index': {'NR': 0}, 'label': {'NR': 'Number'}}},
 'mot_nrg': {'label': 'Motor energy',
  'category': {'index': {'TOTAL': 0, 'PET': 1, 'DIE': 2},
   'label': {'TOTAL': 'Total', 'PET': 'Petroleum products', 'DIE': 'Diesel'}}},
 'engine': {'label': 'Engine capacity of vehicle',
  'category': {'index': {'TOTAL': 0,
    'CC_LT1400': 1,
    'CC1400-1999': 2,
    'CC_GE2000': 3},
   'label': {'TOTAL': 'Total',
    'CC_LT1400': 'Less than 1 400 cm³',
    'CC1400-1999': 'From 1 400 to 1 999 cm³',
    'CC_GE2000': '2 000 cm³ or over'}}},
 'geo': {'label': 'Geopolitical entity (reporting)',
  'category': {'index': {'EU27_2020': 0,
    'BE': 1,
    'BG': 2,
    'CZ': 3,
    'DK': 4,
    'DE': 5,
    'EE': 6,
    'IE': 7,
    'EL': 8,
    'ES': 9,
    'FR': 10,
    'HR': 11,
    'IT': 12,
    'CY': 13,
    'LV': 14,
    'LT': 15,
    'LU': 16,
    '

In [6]:
def decode_dataset_index(flat_index):
    sizes = data["size"]
    dimension_ids = data["id"]
    dimension_data = data["dimension"]


    # Convert to coordinates
    coords = [0] * len(sizes)
    remaining = flat_index
    
    for i in range(len(sizes) - 1, -1, -1):
        coords[i] = remaining % sizes[i]
        remaining = remaining // sizes[i]
    
    # Map to codes
    result = {}
    for i, dim_id in enumerate(dimension_ids):
        coord = coords[i]
        categories = dimension_data[dim_id]['category']
        
        # Find code for this coordinate
        code = next(k for k, v in categories['index'].items() if v == coord)
        
        result[dim_id] = {
            'code': code,
            'label': categories.get('label', {}).get(code, code),
            'index': coord
        }
    
    return result

decode_dataset_index(208)

{'freq': {'code': 'A', 'label': 'Annual', 'index': 0},
 'unit': {'code': 'NR', 'label': 'Number', 'index': 0},
 'mot_nrg': {'code': 'TOTAL', 'label': 'Total', 'index': 0},
 'engine': {'code': 'TOTAL', 'label': 'Total', 'index': 0},
 'geo': {'code': 'CZ', 'label': 'Czechia', 'index': 3},
 'time': {'code': '2016', 'label': '2016', 'index': 46}}

In [7]:
new_passengers_dict = {"MotorEnergy": [], "Engine": [], "Country": [], "Year": [], "NumCars": []}

for key in data["value"].keys():
    key_info = decode_dataset_index(int(key))

    new_passengers_dict["MotorEnergy"].append(key_info["mot_nrg"]["label"])
    new_passengers_dict["Engine"].append(key_info["engine"]["label"])
    new_passengers_dict["Country"].append(key_info["geo"]["label"])
    new_passengers_dict["Year"].append(key_info["time"]["label"])
    new_passengers_dict["NumCars"].append(data["value"][key])

new_passengers_df = pd.DataFrame(new_passengers_dict)
new_passengers_df

,MotorEnergy,Engine,Country,Year,NumCars
0,Diesel,From 1 400 to 1 999 cm³,Albania,2021,233368
1,Diesel,From 1 400 to 1 999 cm³,Albania,2022,252775
2,Diesel,From 1 400 to 1 999 cm³,Albania,2023,277611
3,Diesel,From 1 400 to 1 999 cm³,Austria,1999,1088000
4,Diesel,From 1 400 to 1 999 cm³,Austria,2000,1211000
...,...,...,...,...,...
6760,Total,Total,Kosovo*,2019,309509
6761,Total,Total,Kosovo*,2020,292902
6762,Total,Total,Kosovo*,2021,322701
6763,Total,Total,Kosovo*,2022,339131


In [8]:
total_passengers_df = new_passengers_df[
    (new_passengers_df['Engine'] == 'Total') &
    (new_passengers_df['MotorEnergy'] == 'Total')
]

total_passengers_df

,MotorEnergy,Engine,Country,Year,NumCars
5354,Total,Total,Albania,2013,341691
5355,Total,Total,Albania,2014,378053
5356,Total,Total,Albania,2015,403630
5357,Total,Total,Albania,2016,435613
5358,Total,Total,Albania,2017,417426
...,...,...,...,...,...
6760,Total,Total,Kosovo*,2019,309509
6761,Total,Total,Kosovo*,2020,292902
6762,Total,Total,Kosovo*,2021,322701
6763,Total,Total,Kosovo*,2022,339131


In [20]:
exclude_countries = [
    'European Union - 27 countries (from 2020)',
    'Kosovo*', 
    'United Kingdom'
    'Türkiye'
]

total_passengers_df = total_passengers_df[~total_passengers_df['Country'].isin(exclude_countries)]
total_passengers_df

,MotorEnergy,Engine,Country,Year,NumCars
5354,Total,Total,Albania,2013,341691
5355,Total,Total,Albania,2014,378053
5356,Total,Total,Albania,2015,403630
5357,Total,Total,Albania,2016,435613
5358,Total,Total,Albania,2017,417426
...,...,...,...,...,...
6708,Total,Total,Türkiye,2019,12503049
6709,Total,Total,Türkiye,2020,13099041
6710,Total,Total,Türkiye,2021,13706065
6711,Total,Total,Türkiye,2022,14269352


In [21]:
fig = px.line(total_passengers_df, x = 'Year', y = 'NumCars', title = 'Number of Cars by Year', color = 'Country', 
              color_discrete_sequence = px.colors.qualitative.Bold)
fig.show()

In [11]:
fig = px.bar(total_passengers_df, x = 'Year', y = 'NumCars', title = 'Increase of Passenger Cars Over Time', 
             color_discrete_sequence = px.colors.qualitative.Bold)
fig.show()

fig.write_image("passenger_cars_bar.png")


In [12]:
fig = px.scatter(total_passengers_df, x = 'Year', y = 'NumCars', title = 'Number of Cars over Years',
                 color_discrete_sequence = px.colors.qualitative.Bold,
                 log_x = True, size = 'NumCars')
fig.show()

In [13]:
fig.write_image("passenger_cars.png")


In [14]:
pip install -U kaleido

Note: you may need to restart the kernel to use updated packages.


In [15]:
fig.write_image("passenger_cars.png")


In [16]:
pip install -U kaleido

Note: you may need to restart the kernel to use updated packages.


In [17]:
fig.write_image('passenger_cars_bar.png')